```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Unit Testing Your Web Application}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

I live in Belmont,
in the San Francisco Bay Area Peninsula.
I wish to acknowledge it as the
ancestral homeland
of the
Ramaytush Ohlone people.

In [124]:
import json
import pyramid.response
import pyramid.config
import httpx
from unittest import mock
from hamcrest import assert_that, equal_to
import io

## Pyramid Example

```{=latex}
\begin{frame}
\frametitle{Crash Course in Pyramid}

Some quick examples!

\end{frame}
```

### Static view

```{=latex}
\begin{frame}[fragile]
\frametitle{Static Value}
```

In [19]:
def empty(request):
    return pyramid.response.Response(
        json.dumps({}).encode("ascii"),
        content_type="application/json",
    )
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [15]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Static View Retrieval}
```

In [20]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

### JSON Renderer

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View}
```

In [62]:
def jsonv(request):
    return {}
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(jsonv, route_name="json", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [63]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{JSON View Retrieval}
```

In [64]:
request = client.get("/json")
request.json()

{}

```{=latex}
\end{frame}
```

### Using Parameters

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View}
```

In [27]:
def params(request):
    return dict(thing=request.params["thing"])
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [28]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View Retrieval}
```

In [30]:
request = client.get("/params?thing=hello")
request.json()

{'thing': 'hello'}

```{=latex}
\end{frame}
```

### Using Route Matches

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View}
```

In [38]:
def matches(request):
    return dict(name=request.matchdict["name"])
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [39]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Parameter View Retrieval}
```

In [40]:
request = client.get("/matches/hello")
request.json()

{'name': 'hello'}

```{=latex}
\end{frame}
```

### Using Request Body

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View}
```

In [41]:
def body(request):
    return dict(field=request.json_body["field"])
with pyramid.config.Configurator() as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
    config.add_route("body", "/body")
    config.add_view(body, route_name="body", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [42]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Body View Retrieval}
```

In [43]:
request = client.post("/body", json=dict(field="hello"))
request.json()

{'field': 'hello'}

```{=latex}
\end{frame}
```

### Using Settings

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View}
```

In [44]:
def setting(request):
    return dict(field=request.registry.settings["field"])
with pyramid.config.Configurator(settings=dict(field="field_value")) as config:
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")
    config.add_route("json", "/json")
    config.add_view(json, route_name="json", renderer="json")
    config.add_route("params", "/params")
    config.add_view(params, route_name="params", renderer="json")
    config.add_route("matches", "/matches/{name}")
    config.add_view(matches, route_name="matches", renderer="json")
    config.add_route("body", "/body")
    config.add_view(body, route_name="body", renderer="json")
    config.add_route("setting", "/setting")
    config.add_view(setting, route_name="setting", renderer="json")
    app = config.make_wsgi_app()

```{=latex}
\end{frame}
```

In [45]:
client = httpx.Client(app=app, base_url="https://example.com/")

```{=latex}
\begin{frame}[fragile]
\frametitle{Settings View Retrieval}
```

In [46]:
request = client.get("/setting")
request.json()

{'field': 'field_value'}

```{=latex}
\end{frame}
```

## Pyramid Concepts

### View

```{=latex}
\begin{frame}
\frametitle{View}

Function \pause

Argument: Request \pause

Return value: Response

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route}

Route \pause

to View \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{View Route Predicates}

Route to view can be conditional:\pause

Request type,\pause

Content type,\pause

Arbitrary predicates

\end{frame}
```

### Route

```{=latex}
\begin{frame}
\frametitle{Route}

Path \pause

to route name \pause

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Route: advanced}

\pause

Path fragment matching \pause

...and more

\end{frame}
```

### Registry

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Settings: ad-hoc dictionary

\end{frame}
```

### Configurator

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator}

Routes \pause

Views \pause

Registry \pause

\end{frame}
```

### Pyramid crash course review

```{=latex}
\begin{frame}[fragile]
\frametitle{View Function}
```

In [81]:
def empty(request):
    #     ^^^^^^^
    # A pyramid.request.Request
    
    data = json.dumps({}).encode("ascii")
    #                     ^^^^^^
    #                     Generate bytes
    
    content_type = "application/json"
    
    return pyramid.response.Response(
        data,
        content_type=content_type,
    )

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configuration}
```

In [87]:
with pyramid.config.Configurator() as config:
#^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^    ^^^^^^^
#Commit at the end      ^^^^^^^^^^    ^^^^^^^
#                       Configurator  ^^^^^^^
#                       class         ^^^^^^^
#                                     configurator
#                                     object
    config.add_route("root", "/")
#          ^^^^^^^^^|^^^^^^  ^^^
#          add a new|route   route
#          route    |name    path
    config.add_view(empty, route_name="root")
#          ^^^^^^^^|^^^^^  ^^^^^^^^^^^^^^^^
#          add a   |view   route to attach
#          view to |callable
#          a route |

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{App creation}
```

In [89]:
app = config.make_wsgi_app()
#                 ^^^^
#         Web Standard Gateway Interface

```{=latex}
\end{frame}
```

In [90]:
client = httpx.Client(app=app, base_url="https://example.com/")
request = client.get("/")
request.json()

{}

## Python Web Concepts

### WSGI

```{=latex}
\begin{frame}[fragile]
\frametitle{WSGI}

Framework \pause

to server

\end{frame}
```

### `httpx`

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx}

Requests alternative \pause

Can consume WSGI directly
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Creating the client: \pause
```

In [94]:
client = httpx.Client(
    base_url="https://example.com/",
    app=app,
)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Calling the client: \pause

```

In [98]:
request = client.get("/")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{httpx with WSGI}

Using the httpx request object: \pause

```

In [99]:
request.json()

{}

```{=latex}
\end{frame}
```

## Unit Testing

### Code to unit test

```{=latex}
\begin{frame}[fragile]
\frametitle{System Under Test}
```

In [101]:
def lookup(request):
    return request.registry.settings["matcher"].get(
        request.matchdict["name"],
        # ^^^^^^^^^^^^^^^^^^^^^^^
        # Bug: missing `lower()`
        "default",
    )
def make_app(special_value):
    settings = dict(
        matcher=dict(special=special_value),
    )
    with pyramid.config.Configurator(settings=settings) as config:
        config.add_route("lookup", "/lookup/{name}")
        config.add_view(lookup, route_name="lookup", renderer="json")
    return config.make_wsgi_app()

```{=latex}
\end{frame}
```

### Mocking Request

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}
```

In [112]:
# Setup
request = mock.MagicMock()
request.registry.settings = dict(
    matcher=dict(special="hello"),
)
request.matchdict = dict(name="SPECIAL")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}
```

In [113]:
# Execute
result = lookup(request)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling function directly}
```

In [114]:
# Verify
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

### Calling WSGI

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}
```

In [129]:
# Setup
environ = dict(
    PATH_INFO="/lookup/SPECIAL",
    REQUEST_METHOD="GET",
)
start_response = mock.MagicMock(
    return_value=io.BytesIO()
)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}
```

In [130]:
# Execute
app = make_app("hello")
base_parts = app(environ, start_response)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}
```

In [131]:
# Post-process
parts = [start_response.return_value.getvalue()]
parts.extend(base_parts)
(status, headers), kwargs = start_response.call_args
result = json.loads(b"".join(parts).decode("utf-8"))

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Calling WSGI}
```

In [132]:
# Verify
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

### Using `httpx`

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}
```

In [135]:
# Setup
app = make_app("hello")
client = httpx.Client(
    base_url="https://example.com/",
    app=app,
)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}
```

In [136]:
# Execute
resp = client.get("/lookup/SPECIAL")

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}
```

In [137]:
# Post-process
result = resp.json()

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Using httpx}
```

In [138]:
# Verify
try:
    assert_that(result, equal_to("hello"))
except AssertionError as exc:
    print(exc)


Expected: 'hello'
     but: was 'default'



```{=latex}
\end{frame}
```

## Summary

### Pyramid: Concepts

```{=latex}
\begin{frame}
\frametitle{Summary: Pyramid}

\pause

View: \pause
Function from request to response, \pause
matched to route \pause

Route: \pause
URL to logical route \pause

Registry: \pause
Parameters, shared objects \pause

Configurator: \pause
Views, Routes, Registry \pause

WSGI: \pause
Created from Configurator, \pause
Used by server \pause
(or httpx!)

\end{frame}
```

### Unit Testing: Why?

```{=latex}
\begin{frame}
\frametitle{Summary: Why Unit Test?}

\pause

Verify code, \pause

Quickly, \pause

and safely

\end{frame}
```

### Unit Testing: How?

```{=latex}
\begin{frame}
\frametitle{Summary: How Unit Test?}

\pause

Mock! \pause

Pre-plan \pause

Prefer stable APIs

\end{frame}
```

## Bonus material

### Registry extras

```{=latex}
\begin{frame}
\frametitle{Registry: Application Parameters}

\pause

Utility: Scalable configuration


\end{frame}
```

In [139]:
import zope.interface
import attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Interfaces and Components}
```

In [140]:
class IValueGetter(zope.interface.Interface):
    def get_value(self) -> int:
        ...

@zope.interface.implementer(IValueGetter)
@attrs.frozen
class ValueStorer:
    _value: int
    
    def get_value(self):
        return self._value

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Interface-based Registry}
```

In [141]:
config = pyramid.config.Configurator()
config.registry.registerUtility(ValueStorer(value=42))
config.registry.getUtility(IValueGetter).get_value()

42

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Why Interface-based Registry?}

Namespaced\pause

Semantically meaningful \pause

Configuration

\end{frame}
```

### Configurator extras

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}
```

In [142]:
def root_stuff(config):
    config.add_route("root", "/")
    config.add_view(empty, route_name="root")    
with pyramid.config.Configurator() as config:
    config.include(root_stuff)
    app = config.make_wsgi_app()

In [143]:
client = httpx.Client(app=app, base_url="https://example.com/")

In [144]:
request = client.get("/")
request.json()

{}

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Inclusion}

Include allows: \pause

\begin{itemize}
* Callable \pause
* Callable dotted name \pause
* Module (will use "includeme") \pause
* Module dotted name 

\end{frame}
```

### Scanning

```{=latex}
\begin{frame}[fragile]
\frametitle{Configurator Scanning}

* Scan a package
* Find functions decorated "view config"

\end{frame}
```

```{=latex}
\end{document}
```